## Data preparation for cyclegan
This notebook will allow us to move the right files to prepare cyclegan from one illuminant to another.

In [2]:
import os
import pandas as pd
import numpy
import sys
import shutil

In [21]:
pd.read_csv('./data/train/scene_abandonned_city_54/2500/E/dataset_2500_E.csv').columns


Index(['Unnamed: 0', 'scene', 'illuminant', 'scene_center_x', 'scene_center_y',
       'scene_center_z', 'rendered_image', 'camera_location_x',
       'camera_location_y', 'camera_location_z', 'NW', 'N', 'NE', 'E', 'SE',
       'S', 'SW', 'W', 'white_balance', 'exposure_compensation',
       'pointlight_intensity'],
      dtype='object')

In [46]:
# Desired illuminants {A, B}
color_temperatures = {'A': 2500, 'B': 2500}
locations = {'A': 'SW','B': 'SE'}

destination_dataset_path = './datasets/UELighting/'

train_path = './data/train/'
test_path = './data/test/'
validate_path = './data/validate/'

train_scenes = os.listdir(train_path)
test_scenes = os.listdir(test_path)
validate_scenes = os.listdir(validate_path)


col_names = ['Unnamed: 0', 'scene', 'illuminant', 'scene_center_x', 'scene_center_y',\
             'scene_center_z', 'rendered_image', 'camera_location_x',\
             'camera_location_y', 'camera_location_z', 'NW', 'N', 'NE', 'E', 'SE',\
             'S', 'SW', 'W', 'white_balance', 'exposure_compensation','pointlight_intensity']

dataset_B = pd.DataFrame(columns = col_names)

def generate_dataset(illuminant = 'A', step = 'train', columns = col_names):
    dataset  = pd.DataFrame(columns = col_names)
    for index, scene in enumerate(os.listdir(os.path.join('data', step))):
        data_path = os.path.join('data', step, scene, str(color_temperatures[illuminant]), locations[illuminant])
        dataset_name = 'dataset_' + str(color_temperatures[illuminant]) + '_' + locations[illuminant] + '.csv'
        data = pd.read_csv(os.path.join(data_path, dataset_name))
        # Rename images and copy them to avoid duplicates
        for image_name in data['rendered_image']:
            old_image_name = os.path.join(data_path, image_name)        
            # Separate base from extension
            base, extension = os.path.splitext(image_name)        
            new_image_name = os.path.join(destination_dataset_path, step, illuminant, str(index) + '_' + base + extension)
            shutil.copy(old_image_name, new_image_name)
        data['rendered_image'] = data['rendered_image'].map(lambda image_name:\
                                                            str(index) + '_' + image_name)
        dataset = dataset.append(data)
    dataset.to_csv(os.path.join(destination_dataset_path, 'train', 'A'))
    return dataset

dataset_A = generate_dataset(illuminant = 'A', step = 'train', columns = col_names)
    
    

FileNotFoundError: [Errno 2] File b'data\\train\\scene_moutains_9\\2500\\SW\\dataset_2500_SW.csv' does not exist: b'data\\train\\scene_moutains_9\\2500\\SW\\dataset_2500_SW.csv'